In [2]:
from datasets import Dataset, load_dataset
import random

In [3]:
ds = load_dataset("rasdani/cohere-wikipedia-2023-11-de-top100k-views", split="train")
ds

Dataset({
    features: ['_id', 'url', 'title', 'text', 'views'],
    num_rows: 100000
})

In [20]:
# Convert dataset to pandas DataFrame
df = ds.to_pandas()

# Group by 'title' and filter groups that have 5 or more paragraphs
grouped_df = df.groupby('title', sort=False).size().reset_index(name='count')
filtered_df = grouped_df[grouped_df['count'] >= 5]

# Display the filtered DataFrame
filtered_df


,title,count
0,Deutschland,245
1,Österreich,370
2,Schweiz,417
3,Berlin,282
4,XHamster,35
...,...,...
849,Sunniten,180
850,Passau,187
851,Zoroastrismus,124
852,DisplayPort,35


In [26]:
ds_filtered = ds.filter(lambda x: x['title'] in filtered_df['title'].tolist())
ds_filtered

Dataset({
    features: ['_id', 'url', 'title', 'text', 'views'],
    num_rows: 100000
})

In [27]:
len(set(ds["title"]))

854

In [28]:
seed = int.from_bytes(b"ellamind") % (2**32)
random.seed(seed)

In [29]:
# Define a function to select a random window of 5 consecutive paragraphs
def select_random_window(group):
    if len(group) >= 5:
        start_index = random.randint(0, len(group) - 5)
        return group.iloc[start_index:start_index + 5]
    return None

# Group by 'title', apply the function, and drop groups with less than 5 paragraphs
grouped_df = df.groupby('title', sort=False).apply(select_random_window).dropna()

# Convert the filtered DataFrame back to a dataset
filtered_ds = Dataset.from_pandas(grouped_df.reset_index(drop=True))

# Display the filtered dataset
filtered_ds

/tmp/ipykernel_327603/4200073487.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = df.groupby('title', sort=False).apply(select_random_window).dropna()


Dataset({
    features: ['_id', 'url', 'title', 'text', 'views'],
    num_rows: 4270
})

In [31]:
# Convert dataset to pandas DataFrame
df = filtered_ds.to_pandas()

# Define a function to select a window of 5 consecutive paragraphs and assign scores
def assign_scores(group):
    if len(group) >= 5:
        scores = [0.5, 0.5, 1.0, 0.5, 0.5]  # Scores for the paragraphs
        group['score'] = scores
        return group
    return None

# Group by 'title', apply the function, and drop groups with less than 5 paragraphs
grouped_df = df.groupby('title', sort=False).apply(assign_scores).dropna().reset_index(drop=True)

# Display the DataFrame with the new 'score' column
grouped_df

/tmp/ipykernel_327603/3953193532.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = df.groupby('title', sort=False).apply(assign_scores).dropna().reset_index(drop=True)


,_id,url,title,text,views,score
0,20231101.de_1497705_123,https://de.wikipedia.org/wiki/Deutschland,Deutschland,Der Bundestag machte 1991 Berlin zur Hauptstad...,36234.107805,0.5
1,20231101.de_1497705_124,https://de.wikipedia.org/wiki/Deutschland,Deutschland,Bei der Bundestagswahl 1998 verlor Kohls schwa...,36234.107805,0.5
2,20231101.de_1497705_125,https://de.wikipedia.org/wiki/Deutschland,Deutschland,Die zweite Amtszeit Schröders ab 2002 war von ...,36234.107805,1.0
3,20231101.de_1497705_126,https://de.wikipedia.org/wiki/Deutschland,Deutschland,Angela Merkel beendete die letzte ihrer vier A...,36234.107805,0.5
4,20231101.de_1497705_127,https://de.wikipedia.org/wiki/Deutschland,Deutschland,Der russische Überfall auf die Ukraine 2022 fü...,36234.107805,0.5
...,...,...,...,...,...,...
4265,20231101.de_17999_13,https://de.wikipedia.org/wiki/Nordirland,Nordirland,"Der Haushalt der nordirischen Regierung, der N...",15544.335947,0.5
4266,20231101.de_17999_14,https://de.wikipedia.org/wiki/Nordirland,Nordirland,Es gibt zurzeit keine offizielle Flagge Nordir...,15544.335947,0.5
4267,20231101.de_17999_15,https://de.wikipedia.org/wiki/Nordirland,Nordirland,Lediglich im Sport wird weiter die ehemalige F...,15544.335947,1.0
4268,20231101.de_17999_16,https://de.wikipedia.org/wiki/Nordirland,Nordirland,Die bedeutendsten politischen Parteien in der ...,15544.335947,0.5


In [32]:
ds_upload = Dataset.from_pandas(grouped_df)
ds_upload



Dataset({
    features: ['_id', 'url', 'title', 'text', 'views', 'score'],
    num_rows: 4270
})

In [43]:
ds_upload[4]

{'_id': '20231101.de_1497705_127',
 'url': 'https://de.wikipedia.org/wiki/Deutschland',
 'title': 'Deutschland',
 'text': 'Der russische Überfall auf die Ukraine 2022 führte zu umfassenden Wirtschaftssanktionen des Westens gegen Russland, an denen sich auch Deutschland beteiligte. Unter anderem stoppte Deutschland die Inbetriebnahme der Gaspipeline Nord Stream 2. Die deutsche Wirtschaft, die sich von russischem Gas abhängig gemacht hatte, musste eine starke Teuerung im Energiesektor hinnehmen. Deutschland unterstützte die Ukraine innerhalb des ersten halben Jahres mit Waffen im Wert von mehreren Millionen Euro sowie der Ausbildung ukrainischer Soldaten.',
 'views': 36234.107805126274,
 'score': 0.5}

In [44]:
ds_upload.push_to_hub("rasdani/cohere-wikipedia-2023-11-de-scores")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rasdani/cohere-wikipedia-2023-11-de-scores/commit/2f9abf9cd3c51e0c959f195a777a51d7084d5860', commit_message='Upload dataset', commit_description='', oid='2f9abf9cd3c51e0c959f195a777a51d7084d5860', pr_url=None, pr_revision=None, pr_num=None)